<a href="https://colab.research.google.com/github/ailab-nda/NLP/blob/main/Japanese_GTP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 日本語GTPによる自然言語語処理

## 背景 
最近のNLPに関する研究では、モデルのパラメータ数が多いほど高い性能を発揮することが知られています。実際、英語においては、人間が作る文章とAIが作る文章の区別が付かないほどですが、日本語はその特殊性から英語ほど研究が進んでいません。そこで本演習では、現在フリーで使用できるモデルのうち、最も大規模な13億パラメータを持つ日本語に特化したGPT言語モデルを使用し、日本語における自然言語処理の現状を認識してもらいます。


## 準備１（関連ライブラリのインストール）

In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install datasets

## 準備２（関連ライブラリのインポート）

In [ ]:
import torch
import textwrap
from transformers import T5Tokenizer, AutoModelForCausalLM
import re

## 準備３（GTPモデルのダウンロード）

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt-1b")
model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt-1b")

## 準備４（GPUの使用）

In [ ]:
if torch.cuda.is_available():
    model = model.to("cuda")

# 演習

## 1. 文章の生成
書き出しを入力することで、続きの文章を生成します。

### 文章を生成する関数の定義
gtp モデルの generate 関数を呼び出すことで行います。

In [ ]:
def generate_sentense(question):
    token_ids = tokenizer.encode(text, add_special_tokens=False, return_tensors="pt")
    with torch.no_grad():
        output_ids = model.generate(
            token_ids.to(model.device),
            max_length=100,
            min_length=100,
            do_sample=True,
            top_k=500,
            top_p=0.95,
            pad_token_id=tokenizer.pad_token_id,
            bos_token_id=tokenizer.bos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            bad_word_ids=[[tokenizer.unk_token_id]]
        )
    output = tokenizer.decode(output_ids.tolist()[0])
    return output

### 文章生成の例１

In [ ]:
text = "私が御社を志望したのは、"
answer = generate_sentense(text)
print(answer)

### 文書生成の例２
上記の text の内容を変えて、違う書き出しで文章を作成させてみましょう。

In [ ]:
text = "昨日、近所の吉野家行ったんです。吉野家。そしたらなんか人がめちゃくちゃいっぱいで座れないんです。" # ここを自分の例にする
answer = generate_sentense(text)
print(answer)

## 2. 質問とその答え
文章生成関数をちょっと作り替えると、質問に対する答えを生成してくれる関数が作れます。

### 質疑応答用関数
上で作った generate_sentense の出力の余計な部分をカットするだけです。

In [ ]:
def q_and_a(question):
    output = generate_sentense(question)
    output2 = output[len(text):]
    return(output2[:re.search('。|\?|\!', output2).start()+1])

### 例１：テキストの分類
例をいくつか挙げて、最後に聞いたものがどちらに当てはまるかを答えさせます。

In [ ]:
text = "「最悪」はネガティブな言葉。「いいね」はポジティブな言葉。「素晴らしい」はポジティブな言葉。「良くはない」は"
answer = q_and_a(text)
print(answer)

### 例２：翻訳
英語と日本語の対を与えて、最後の英語がどのような日本語に対応するかを答えさせます。

In [ ]:
text = "Helloは、こんにちはという意味です。Good morningは、おはようという意味です。Thank youは、"
answer = q_and_a(text)
print(answer)

### 例３：やってみよう
上記の text の内容を変えて、違う例を作成してみましょう。

In [ ]:
text = "我輩は「猫」である。我輩は「犬」である。正しいのは" # ここを自分の例にする
answer = q_and_a(text)
print(answer)